# How to extract a custom feature with Damavand?

## 1. Feature Extraction by Damavand

Hand-crafted features developed by experts are of great importance for rotating machinery condition monitoring. Damavand simplifies the extraction of a wide range of [features](https://github.com/amirberenji1995/damavand/blob/main/documentations/signal_processing.md#features-to-extract). through the employment of a Python function as [`feature_extractor`](https://github.com/amirberenji1995/damavand/blob/main/documentations/signal_processing.md#feature_extractorsignals-features). This function takes a collection of features as key-value pairs of the following format:

```python
features = {
  'feature_name': (feature, (args), {kwargs})
}
```
In this notation, `feature`, `args` and `kwargs` are the feature implemented as a Python function, tuple of its positional arguments and the dict of keyword arguments.

Once such dictionary is declared, the `feature_extractor` function can be used easily as below:

```python
from numpy import mean, std
from damavand.damavand.signal_processing.feature_extraction import rms, feature_extractor

features = {
  'mean': (mean, (), {}),
  'std': (std, (), {}),
  'rms': (rms, (), {}),
}

# Assuming the desired signals are stored in the `signals` variable
features_df = feature_extractor(signals, features)
```



## 2. Extraction of a custom feature

The application of `feature_extractor` is not limited to the features available in Damavand; custom features can be implemented (as Python functions) and extracted by this function. To do so, let's implement the following feature (known as Squared Mean of Square Roots of Absolutes or SMSA) as a Python function:

$f = \left(\frac{\sum_{n=1}^{N} \sqrt{\|x(n)\|}}{N}\right)^{2}$

```python
import numpy
from damavand.damavand.signal_processing.feature_extraction import feature_extractor

def smsa(x):
  return np.square(np.mean(np.sqrt(np.abs(x))))

features = {
  'smsa': (feature, (), {}),
}

# Assuming the desired signals are stored in the `signals` variable
features_df = feature_extractor(signals, features)
```

The above function takes in no argument, except for the signal itself; however, it is not always the case. Consider the following feature (known as spectral centroid or SC for short):

$f = \frac{\sum_{k=1}^{K} f_k \cdot s(k)}{\sum_{k=1}^{K} s(k)}$

where $s(k)$ and $f_k$ correspond to the frequency spectrum and corresponding frequency axis. In contrast to the previous feature, this feature also requires the frequency axis as an additional argument. The below implementation makes the extraction of this feature, possible:

```python
import numpy as np
from damavand.damavand.signal_processing.feature_extraction import feature_extractor

def feature(spectrum, freq_axis):
  return np.sum(spectrum * freq_axis) / np.sum(spectrum)

# Assuming that the `freq_axis` stores the frequency axis for detail on how to achieve this checkout:
# https://github.com/amirberenji1995/damavand/blob/main/documentations/utils.md#fft_freq_axistime_len-sampling_freq
features = {
  'sc': (feature, (freq_axis), {})
}

# Assuming that the `signals_fft` includes frequency domain signals
features_df = feature_extractor(signals_fft, features)
```